****
## Модуль django.contrib.auth

Работа с пользователями в Django обеспечивается взаимодействием нескольких компонентов — модулей и промежуточных слоёв (middlewares), по умолчанию подключённых к проекту. 

«Ядро» системы работы с пользователями — это встроенный модуль `django.contrib.auth`, именно его мы и будем изучать в этом и ближайших уроках.

Модуль `django.contrib.auth` автоматически подключается при развёртывании Django-проекта. Этот модуль исходно добавлен в список подключённых приложений `INSTALLED_APPS` в *settings.py*:

```py
# acme_project/settings.py
...

INSTALLED_APPS = [
    ...,
    'django.contrib.admin',
    'django.contrib.auth',   # Приложение для работы с пользователями.
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    ...,
] 
```

Модуль `django.contrib.auth` прямо «из коробки» готов к работе: в нём есть свои модели, свой urls.py и views.py, набор HTML-шаблонов для отображения страниц с формами. 

К основным возможностям модуля относятся:

* логин (вход в систему) и логаут (выход из системы) пользователей,

* смена и сброс пароля,

* управление пользователями,

* проверка прав доступа пользователей.

Помимо `django.contrib.auth`, для работы с пользователями применяются и другие модули; узнать о них подробнее можно [в документации](https://docs.djangoproject.com/en/3.2/topics/auth/#installation).

Подробное описание полей модели User есть [в документации](https://docs.djangoproject.com/en/3.2/ref/contrib/auth/#django.contrib.auth.models.User); часть из них мы рассмотрим повнимательнее.

***
## Расширение модели пользователя

Для расширения модели пользователя создают дополнительную модель, в которую добавляют поля, которых не хватает в основной модели пользователя. Эту модель связывают с основной моделью пользователя через связь «один-к-одному».

У этого способа есть недостатки:

* нагрузка на БД повысится, ведь для получения дополнительных полей придётся отправлять запросы с JOIN; запросы к связанной модели нужно будет оптимизировать;

* придётся дополнительно настроить админку так, чтобы поля основной и дополнительной моделей отображались на одной странице, а не были разбросаны по разным разделам;

* нужно будет описать автоматическое создание объекта кастомной модели при создании объекта в «главной» модели пользователя.

Однако этот подход хорош тем, что его можно применить в любой момент, даже если проект уже разрабатывается, а база данных наполнена.

Подробнее о расширении модели пользователя можно почитать [в документации](https://docs.djangoproject.com/en/3.2/topics/auth/customizing/#extending-the-existing-user-model).

***
## Замена модели пользователя

Стандартная модель пользователя `User`, которая по умолчанию применяется в Django, унаследована от модели `AbstractUser` из модуля `django.contrib.auth.models`. 

`AbstractUser` — это абстрактный класс, полная модель пользователя со всеми необходимыми полями.

Можно создать собственную модель пользователя, унаследовав её от `AbstractUser`, и добавить в модель-наследник необходимые дополнительные поля. 

В дальнейшем собственную модель пользователя можно будет расширять и как угодно изменять: управление этой моделью будет полностью в руках разработчика. А про стандартную модель `User` можно будет просто забыть. 

Однако при таком подходе есть определённые ограничения: заменять модель пользователя рекомендуется на самом старте проекта, до выполнения встроенных миграций.

Если попробовать заменить модель пользователя уже после применения встроенных миграций, то возникнут ошибки при выполнении пользовательских миграций: существующие в БД таблицы ссылаются на другую (встроенную) модель пользователя, и автоматически разрешить этот конфликт нельзя.

Есть и радикальный способ, он сработает даже в том случае, если БД наполнена: создать собственную модель пользователя, удалить базу данных, а затем создать и выполнить миграции. Разумеется, при таком подходе без дополнительных усилий не удастся сохранить данные в БД.

Самый простой вариант замены пользовательской модели при старте проекта выполняется в три шага:

1. Создать кастомную модель пользователя, унаследовав её от `AbstractUser`.

2. Указать ссылку на кастомную модель в константе `AUTH_USER_MODEL` в настройках проекта.

3. Зарегистрировать новую модель в админке.

Для моделей и других сущностей, управляющих пользователями, можно создать собственное приложение. Но это не догма, модель пользователя может храниться где-то ещё, по выбору разработчика.

Файлы, относящиеся к управлению пользователями, принято хранить в отдельном приложении; традиционно его называют **users**.

Первым делом создаётся и регистрируется приложение **users**, а в нём описывается новый класс — кастомная модель пользователя; этот класс следует создать в самом начале работы над проектом, до того, как будут выполнены встроенные миграции.

В новую модель пользователя сразу добавим поле `bio`.

```py
# users/models.py
from django.contrib.auth.models import AbstractUser
from django.db import models


class MyUser(AbstractUser):
    bio = models.TextField('Биография', blank=True) 
```

Теперь нужно внести правки в *settings.py*: здесь надо указать, что вместо стандартной модели пользователя будет использоваться кастомная модель (в нашем случае — из приложения **users**).

```py
# settings.py
...
AUTH_USER_MODEL = 'users.MyUser' 
```

После этого надо создать и применить миграции.

Модель пользователя нужно зарегистрировать в админке, иначе управлять пользователями через админку будет невозможно. 

Чтобы поле с биографией отображалось в форме для редактирования пользователя, надо расширить атрибут `fieldsets` (набор полей) класса `UserAdmin`.

```py
# users/admin.py
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin

from .models import MyUser

# Добавляем поле с биографией 
# к стандартному набору полей (fieldsets) пользователя в админке.
UserAdmin.fieldsets += (
    # Добавляем кортеж, где первый элемент — это название раздела в админке,
    # а второй элемент — словарь, где под ключом fields можно указать нужные поля.
    ('Extra Fields', {'fields': ('bio',)}),
)
# Регистрируем модель в админке:
admin.site.register(MyUser, UserAdmin) 
```

Более подробно про атрибут fieldsets можно почитать [в документации](https://docs.djangoproject.com/en/3.2/ref/contrib/admin/#django.contrib.admin.ModelAdmin.fieldsets).

В случае, если новые поля в модели не нужны, но разработчик решил последовать рекомендациям Django и создать собственную модель пользователя — можно создать пустой класс, унаследованный от `AbstractUser`:

```py
from django.contrib.auth.models import AbstractUser


class MyUser(AbstractUser):
    pass 
```

Эта модель точно так же должна быть указана в *settings.py* в константе `AUTH_USER_MODEL`. А при добавлении модели в админку нет необходимости указывать новые поля:

```py
# users/admin.py
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin

from .models import MyUser 

# Не добавляем поля через UserAdmin.fieldsets,
# а сразу регистрируем модель в админке:
admin.site.register(MyUser, UserAdmin) 
```

Если в какой-то момент развития проекта потребуется добавить новые поля в модель пользователя — нужно будет:  

* добавить поля в собственный класс пользователя;

* создать и выполнить миграции;

* через `UserAdmin.fieldsets` вывести новые поля пользователя в админ-зону.

***
## Как обратиться к модели пользователя

В Django-проекте модель пользователя по умолчанию называется User; разработчик может создать кастомную модель пользователя и указать эту кастомную модель в качестве основной модели пользователя в проекте.

Если к моменту создания кастомной модели в проекте уже есть какой-то код, работающий с моделью пользователя, — этот код придётся изменять: раньше код обращался к встроенной модели `User`, а теперь нужно будет обращаться к кастомной модели с именем, например, `MyUser`. Придётся перебирать весь код и вносить правки. Нехорошая перспектива.

```py
# Например, код был таким:
result = User.objects.all()

# Но разработчик изменил модель пользователя на кастомную MyUser;
# придётся подправить код:
result = MyUser.objects.all()

# И так по всему проекту. 
```

Разработчики Django [рекомендуют](https://docs.djangoproject.com/en/2.2/topics/auth/customizing/#referencing-the-user-model) всегда обращаться к модели пользователя не напрямую, а через функцию `get_user_model()`, даже если модель пользователя не изменена (кто знает, когда понадобится её изменить! Лучше заранее подготовиться к этому).

Функция `get_user_model()` возвращает ту модель, которая зарегистрирована в качестве основной модели пользователя в *settings.py*.

```py
# Получаем модель, зарегистрированную в конфиге проекта,
# в константе AUTH_USER_MODEL
User = get_user_model()

# И в коде применяем значение переменной User, 
# которое вернула функция get_user_model():
result = User.objects.all() 
```

Если разработчик повсюду предусмотрительно применял функцию `get_user_model()` — при замене модели пользователя не придётся вносить изменения по всему проекту: вместо прежней модели функция будет возвращать новую.

* Подробности о модели `AbstractUser` всегда можно подсмотреть [в документации](https://docs.djangoproject.com/en/3.2/topics/auth/customizing/#django.contrib.auth.models.AbstractUser).

* Функция `get_user_model()` тоже описана в документации, [вот здесь](https://docs.djangoproject.com/en/3.2/topics/auth/customizing/#django.contrib.auth.get_user_model).

***
## Класс AbstractBaseUser

Модель пользователя можно перестроить значительно глубже: например, убрать из неё ненужные поля, выбрать для логина поле `email` вместо `username`, а можно и полностью переписать весь процесс аутентификации. 

Для этого кастомную модель пользователя надо наследовать не от `AbstractUser`, а от базовой модели `AbstractBaseUser` из модуля `django.contrib.auth.base_user`. 

В этом классе прописаны лишь несколько основных полей — например, пароль и время последнего логина, а также [базовые методы](https://docs.djangoproject.com/en/3.2/topics/auth/customizing/#django.contrib.auth.models.AbstractBaseUser) модели пользователя (связанные в основном с обработкой пароля и его сбросом). Все остальные поля и методы надо будет настроить самостоятельно. Как следствие — этот вариант значительно сложнее других вариантов переопределения модели пользователя.

Подробнее о настройке такой кастомной модели пользователя можно почитать [в документации](https://docs.djangoproject.com/en/3.2/topics/auth/customizing/#specifying-a-custom-user-model).